# AxiomaPy Direct Access


All API access is managed through a series of modules and classes using the active session.

In some cases direct API access may be preferred or necessary.  
The following examples show how to access the api classes and perform some actions on portfolios.


## Imports

The axiomapy imports.
Each 'section' of the API is available to import from the axiomaapi package.

In [ ]:
from axiomapy import AxiomaSession, AxiomaResponse
from axiomapy.axiomaapi import (
    PortfoliosAPI,
    AnalysesPerformanceAPI,
    AnalysesRiskAPI
)


Some other imports

In [ ]:
from pprint import pprint
from datetime import datetime, timedelta
import logging

In [ ]:
import logging

logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s %(levelname)-8s %(message)s', 
                    datefmt='%Y-%m-%d %H:%M:%S')

## Credentials and Creating an Active Sesssion
Load up some credentials to work with (there is a sample.credentials.json in the ./examples/credentials folder, copy this file to credentials.json and update. Alternatively skip the next set of code and set the credentials directly when creating the session):

In [ ]:
from axiomapy.examples.load_credentials import get_user
user1 =  get_user('user1')

In [ ]:
# if the credentials json is not used enter credentials directly below:
AxiomaSession.use_session(user1['client_id'], user1['username'], user1['password'], user1['domain'], application_name='With analyses examples connection' )
# test the connection
me = AxiomaSession.current.test()

## Get a list of portfolios

In [ ]:
portfolios_response = PortfoliosAPI.get_portfolios(top=10)


The response object contains subsets of the original response

In [ ]:
print(f"Returned type is {type(portfolios_response)}")

In [ ]:
pprint(portfolios_response.json())

you can still access the original response

In [ ]:
original_response = portfolios_response.response

However, the original reponse can be requested

In [ ]:
response = PortfoliosAPI.get_portfolios(top=10, return_response=True)

print(f"The returned type is {type(response)}")

print(f"Access the original request url info {response.request.url}")

## Performance Attribution Request

A Performance attribution request is to analyse the performance of a portfolio during given dates

A pre-step to main PA task is to starts a job to pre-calculate the analytics results that form the inputs to performance attribution.

In [ ]:
precompute_param = {
    "asOfDate": "2021-03-31", 
    "benchmark": {"useNoBenchmark": True}, 
    "performanceAttributionSettingsId": 2136463,
    "computingBehavior":"MissingOnly"
}

In [ ]:
analyses_test = AnalysesPerformanceAPI.post_precompute(portfolio_id=123330, precompute_parameters=precompute_param)

Once precompute is done for all the dates for which PA task has to be performed, the PA task can be triggered.

The PA task body accepts the start and end dates for PA along with the Performance Attribution settings ID

In [ ]:
pa_params = {
    "startDate": "2021-03-30", 
    "endDate": "2021-03-31",
    "benchmark": {"useNoBenchmark": True}, 
    "performanceAttributionSettingsId": 2136463,
    "computingBehavior":"MissingOnly"
}

The header of the response to the request consists of the request id which is used in further requests

In [ ]:
pa_task = AnalysesPerformanceAPI.post_performance_analysis(portfolio_id=123330, analyses_parameters=pa_params)
requestId = int(pa_task.headers["location"].split("/")[-2])

The status of the request and logs can be attained using the request id

In [ ]:
status = AnalysesPerformanceAPI.get_status(request_id=requestId)
logs = AnalysesPerformanceAPI.get_logs(request_id=requestId)

## Risk Model Service

The Risk Model Service is helpful in generating Risk Model based on the exposures of a portfolio or list of instruments. It can also be used to output factor returns for a given portfolio or list of instruments.

Create a request with the desired parameters: portfolio or instruments; risk model definition and statistic definition

In [ ]:
rms_parameters = {
  'portfolioId': 123442,
  'analysisDate': '2020-09-15',
  'riskModelDefinitionId': 3707082
}

The header of the response to the request consists of the request id which is used in further requests

In [ ]:
req = AnalysesRiskAPI.post_risk_model_request(risk_model_parameters=rms_parameters)
requestId = int(req.headers["location"].split("/")[-1])

The status of the request can be fetched using the request id. Once the request is completed the user can also access the logs for the same. 

In [ ]:
status = AnalysesRiskAPI.get_risk_model_request_status(requestId)

logs = AnalysesRiskAPI.get_risk_model_logs(requestId)

The user can get the results of the request using the request id. The user can choose the results to be in raw format or refined by changing the parameter show_raw_results in the request

In [ ]:
results = AnalysesRiskAPI.get_risk_model_results(request_id=requestId, show_raw_results=False)

The risk model data from the above step can also fetched in zipped format. 

In [ ]:
headers={"Accept":"multipart/x-zip"}
raw_results = AnalysesRiskAPI.get_risk_model_results(request_id=requestId, headers=headers)

This data can be extracted and saved to a local directory "C:/RMS/temp" with a sample code below

In [ ]:
import zipfile
from io import BytesIO
z = zipfile.ZipFile(BytesIO(raw_results.content))
for name in z.namelist():
    z.extract(name, "C:/RMS/temp/")